# 电子积分准备：傅里叶变换

重新初始化一下我们的体系。

In [2]:
from pyscf.pbc import gto

cell = gto.Cell()
cell.atom='''
C 0.000000000000   0.000000000000   0.000000000000
C 1.685068664391   1.685068664391   1.685068664391
'''
cell.a = '''
0.000000000, 3.370137329, 3.370137329
3.370137329, 0.000000000, 3.370137329
3.370137329, 3.370137329, 0.000000000'''
cell.unit = 'B'
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.build()

In [3]:
S_ref = cell.pbc_intor('int1e_ovlp')

## 重叠积分：傅里叶变换

$$ \phi(\boldsymbol{G}) = \int e^{-i \boldsymbol{G}\cdot\boldsymbol{r}} \phi(\boldsymbol{r}) \mathrm{d} \boldsymbol{r} $$

In [4]:
mesh = [10,10,10]
grids = cell.get_uniform_grids(mesh)

In [5]:
Gv = cell.get_Gv(mesh=mesh)
print(Gv.shape)

(1000, 3)


## 电子-核吸引能积分



实际上下式的第二个等号并不严格，只是代表了周期性程序的通常处理方式。FFT $\{V_\text{nuc}\}$ 本来是发散的，利用Ewald求和可以避免计算 $G=0$ 的项 (Martin Appendix F)，并将 $E_{\text{nuc}}$ 分为三部分，另两部分在其他地方计入。

$$ \langle \phi_\mu | V_{\text{nuc}} | \phi_\nu \rangle = \dfrac{1}{N} \sum_{L L'} \int \mu^*(\boldsymbol{r} - \boldsymbol{L}) \nu(\boldsymbol{r} - \boldsymbol{L}') \sum_A \dfrac{Z_A}{| \boldsymbol{r} - \boldsymbol{R}_A|} \mathrm{d} \boldsymbol{r} \\
= \sum_{G \neq 0} \omega_G \rho_{\mu\nu}(-\boldsymbol{G}) V_{\text{nuc}}(\boldsymbol{G}) \; \text{(Martin 12.4)} $$

$$ \rho_{\mu\nu}(\boldsymbol{G}) = \sum_{L L'} \int e^{-i\boldsymbol{G}\cdot\boldsymbol{r}} \mu^*(\boldsymbol{r} - \boldsymbol{L}) \nu(\boldsymbol{r} - \boldsymbol{L}') $$

$$ V_{\text{nuc}}(\boldsymbol{G}) = \dfrac{4\pi}{G^2} \sum_A Z_A e^{-i\boldsymbol{G}\cdot\boldsymbol{R}_A} $$

In [6]:
from pyscf.pbc.tools import fft, ifft, cutoff_to_mesh
from pyscf.pbc import df
import numpy as np

def get_nuc_ref(cell, mesh):
    mydf = df.FFTDF(cell).set(mesh=mesh)
    return mydf.get_nuc()

def get_nuc(cell, mesh, grids):
    R = cell.atom_coords()
    Gv = cell.get_Gv(mesh=mesh)
    SI = np.exp(-1j*np.einsum('zs,xs->zx', R, Gv))
    rho_nuc_G = -np.einsum('z,zx->x', cell.atom_charges(), SI)

    aoR = cell.pbc_eval_gto('GTOval', grids)
    nao = aoR.shape[1]
    pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    pair_ao_G_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                           mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    #print(G2)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    v = w * np.einsum('x,x,xij->ij', rho_nuc_G, coulG, pair_ao_G_space)
    return v

enuc = get_nuc(cell, mesh, grids)
enuc_ref = get_nuc_ref(cell, mesh)
print( np.linalg.norm(enuc - enuc_ref))
#ke_cutoff = 20.0
#_mesh = cutoff_to_mesh(cell.lattice_vectors(), ke_cutoff)
#print(_mesh)
#_grids = cell.get_uniform_grids(_mesh)
#enuc_cut = get_nuc(cell, _mesh, _grids)
#print(np.linalg.norm(enuc_cut - enuc))

0.006251361039666754


/tmp/ipykernel_1555449/2374552966.py:24: RuntimeWarning: divide by zero encountered in divide
  coulG = 4 * np.pi / G2


## ERI

In [7]:
def get_eri(cell, mesh, grids):
    Gv = cell.get_Gv(mesh=mesh)

    aoR = cell.pbc_eval_gto('GTOval', grids)
    nao = aoR.shape[1]

    pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    weight = cell.vol / np.prod(mesh)
    pair_ao_G_space = fft(pair_ao_real_space.reshape(-1,nao**2).T,
                          mesh).T.reshape(-1,nao,nao) * weight
    pair_ao_G_inv_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                               mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    eri = w * np.einsum('x, xij, xkl -> ijkl', coulG, pair_ao_G_space, pair_ao_G_inv_space)
    return eri

def get_eri_ref(cell, mesh):
    nao = cell.nao_nr()
    mydf = df.FFTDF(cell).set(mesh=mesh)
    eri = mydf.get_eri(compact=False).reshape(nao,nao,nao,nao)
    return eri

eri = get_eri(cell, mesh, grids)
print(abs(eri - get_eri_ref(cell, mesh)).max())

/tmp/ipykernel_1555449/1782321194.py:16: RuntimeWarning: divide by zero encountered in divide
  coulG = 4 * np.pi / G2


2.197256950963243e-06
